In [1]:
import time
import zipfile
import json
import pandas as pd
import os
import requests
from datetime import date
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

path ="htp/chrome.exe"
service=Service(executable='./chromedriver.exe')
driver = webdriver.Chrome(service=service)
keyword="allowed-amounts"

In [2]:
def parsess(json_data):
    allowed_amounts_data=[]
    for summary in json_data:
        for item in summary["out_of_network"]:
            for amount in item["allowed_amounts"]:
                for payments in amount["payments"]:
                    for provider in payments["providers"]:
                        for npi in provider["npi"]:
                            allowed_amounts_data.append({
                                "Reporting_entity_name" : summary["reporting_entity_name"],
                                "Reporting_entity_type" : summary["reporting_entity_type"],
                                "Plan_name" : summary["plan_name"],
                                "Plan_id_type":summary["plan_id_type"],
                                "Plan_id":summary["plan_id"],
                                "Plan_market_type":summary["plan_market_type"],
                                "Last_update_on":summary["last_update_on"],
                                "Version":summary["version"],
                                "Name":item["name"],
                                "Billing Code":item["billing_code"],
                                "Description":item["description"],
                                "TIN Type":amount["tin"]["type"],
                                "TIN Value":amount["tin"]["value"],
                                "Service Code":",".join(amount["service_code"]),
                                "Billing Class":amount["billing_class"],
                                "Allowed Amounts":payment["allowed_amount"],
                                "Billed Charge":amount["billing_charge"],
                                "NPI":npi,
                                "Flag":1,
                                "Initial_update":date.today(),
                                "Final_update":date.today()
                            })
    return allowed_amounts_data

In [3]:
def to_excel(summary_main):
    df1=pd.read_excel("C:/Users/gopik/OneDrive/Desktop/keerthicam/HPT-main/Meta_Data.xlsx",sheet_name="Data_File")
    df2=pd.DataFrame(summary_main)
    new_data_frame=pd.DataFrame(columns=df1.columns)
    for index,row in df2.iterrows():
        if(df1==row).all(axis=1).any():
            df1.loc[(df1==row).all(axis=1),'Latest_Update'] = date.today()
        else:
            new_dataframe.loc[len(new_data_frame.index)]=row
    df1=pd.concat([df1,new_data_frame],ignore_index=True)
    return df1

In [4]:
def create_metadata():
    if not os.path.isfile("C:/Users/gopik/OneDrive/Desktop/keerthicam/HPT-main/Meta_Data.xlsx"):
        df=pd.DataFrame(columns=["Reporting_entity_name","Reporting_entity_type",
                               "Plan_name","Plan_id_type","Plan_id","Plan_market_type",
                               "Last_update_on","Version","Name","Billing Code","Description",
                               "TIN Type","TIN Value","Service Code","Billing Class","Allowed Amounts",
                               "Billed Charge","NPI","Flag","Initial_update","Final_update"])
        with pd.ExcelWriter('C:/Users/gopik/OneDrive/Desktop/keerthicam/HPT-main/Meta_Data.xlsx',engine='openpyxl') as writer:
            df.to_excel(writer,sheet_name='Data_File',index=False)

In [5]:
def extractor(lists):
    file_count=0
    print("Total_file",len(lists))
    result_main=[]
    c=0
    for lst in lists:
        link_element = lst.find('a')
        if link_element:
            link_url = link_element.get('href')
            response = requests.get(link_url)
            filename = link_url.split('/')[-1]
            full_path = os.path.join("C:/Users/gopik/OneDrive/Desktop/keerthicam/HPT-main/zips", filename)
            with open(full_path, 'wb') as file:
                file.write(response.content)
            with zipfile.ZipFile(full_path, 'r') as zip_ref:
                zip_ref.extractall("C:/Users/gopik/OneDrive/Desktop/keerthicam/HPT-main/extractss")
            result=json_parser(filename)
            if result:
                for i in result:
                    result_main.append(i)
            file_count+=1
            print('Completed-->{0}/{1}'.format(file_count,len(lists)))
            c+=1
            if c==5:
                break
        else:
            print("No <a> tag found in list item.")
    return result_main

In [6]:
def json_parser(filename):
    jsons = 'C:/Users/gopik/OneDrive/Desktop/keerthicam/HPT-main/extractss/'+filename.split('.')[0] +'.json'
    file=open(jsons)
    data=json.load(file)
    summary={}
    allowed_amount=parses([data])
    return allowed_amount

In [7]:
def scrapper(url):
    print("Scrpping Started")
    driver.get(url)
    time.sleep(10)

    search=driver.find_element(By.CLASS_NAME,"ant-input")
    search.send_keys(keyword)
    search.send_keys(Keys.RETURN)
    time.sleep(3)

    page_source=driver.page_source
    soup=BeautifulSoup(page_source,'html.parser')
    lists=soup.find_all('li',class_='ant-list-item')
    return lists

In [8]:
if __name__ == "__main__":
    start_time = time.time()
    url = "https://transparency-in-coverage.optum.com/"
    response=requests.get(url)
    result=[]
    
    if response.status_code==200:
        create_metadata()
        lists=scrapper(url)
        results=extractor(lists)
        dff=to_excel(results)
        driver.quit()
    else:
        print("Error accessing page or no URLs processed.")
    print("Execution_time: ---%s seconds ---" % ((time.time()-start_time)))

Scrpping Started
Total_file 574
Completed-->1/574
Completed-->2/574
Completed-->3/574
Completed-->4/574
Completed-->5/574
Execution_time: ---40.93163728713989 seconds ---
